# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
from skimage.transform import resize
import datetime
import os
import warnings
warnings.filterwarnings('ignore')

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D,Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import SimpleRNN
from keras.layers import Dense, Activation
from keras.layers import LSTM,Dropout,GlobalAveragePooling2D,GlobalAveragePooling3D

Using TensorFlow backend.


We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 30
image_len=100;
image_wid=100;
print(train_doc[:10])

['WIN_20180926_16_54_08_Pro_Right_Swipe_new;Right_Swipe_new;1\n'
 'WIN_20180925_18_02_58_Pro_Thumbs_Down_new;Thumbs_Down_new;3\n'
 'WIN_20180925_17_33_08_Pro_Left_Swipe_new;Left_Swipe_new;0\n'
 'WIN_20180925_17_51_17_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n'
 'WIN_20180926_17_17_35_Pro_Left_Swipe_new;Left_Swipe_new;0\n'
 'WIN_20180926_17_30_47_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n'
 'WIN_20180926_17_29_23_Pro_Stop_new;Stop_new;2\n'
 'WIN_20180926_17_12_26_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n'
 'WIN_20180926_17_29_01_Pro_Stop_new;Stop_new;2\n'
 'WIN_20180907_16_32_11_Pro_Stop Gesture_new;Stop Gesture_new;2\n']


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [*range(0,30,2)]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size;
#         print("Total batches:"+str(num_batches))
        for batch in range(num_batches): # we iterate over the number of batches
#             print("In batch"+str(batch))
            batch_data = np.zeros((batch_size,len(img_idx),image_len,image_wid,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
#                 print("index accessed:"+str(folder + (batch*batch_size)))
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])# read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in

                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape
                    image = imresize(image,(image_len,image_wid))
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if len(t)%batch_size==0: continue
#         print("in part-2")
        batch_size2 = len(t) - batch_size*num_batches
#         print("batchSize2: "+str(batch_size2))
        batch_data = np.zeros((batch_size,len(img_idx),image_len,image_wid,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
        for folder in range(batch_size2): # iterate over the batch_size
#             print("index accessed:"+str(folder + (num_batches*batch_size)))
            imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape
                image = imresize(image,(image_len,image_wid))
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                batch_data[folder,idx,:,:,0] = image[:,:,0]/255
                batch_data[folder,idx,:,:,1] = image[:,:,1]/255
                batch_data[folder,idx,:,:,2] = image[:,:,2]/255
                    
            batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
        yield batch_data, batch_labels


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [5]:
g = generator('./Project_data/val',val_doc,30)
print("-------------Iteration-1---------------")
next(g)
print("-------------Iteration-2---------------")
next(g)
print("-------------Iteration-3---------------")
next(g)
print("-------------Iteration-4---------------")
next(g)
print("-------------Iteration-5---------------")
next(g)
print("-------------Iteration-6---------------")
next(g)
print("-------------Iteration-7---------------")
next(g)

-------------Iteration-1---------------
Source path =  ./Project_data/val ; batch size = 30
-------------Iteration-2---------------
-------------Iteration-3---------------
-------------Iteration-4---------------
-------------Iteration-5---------------
-------------Iteration-6---------------
-------------Iteration-7---------------


(array([[[[[0.37647059, 0.14509804, 0.08235294],
           [0.38823529, 0.15686275, 0.08627451],
           [0.40392157, 0.16078431, 0.08235294],
           ...,
           [0.43921569, 0.28235294, 0.19607843],
           [0.43137255, 0.2745098 , 0.18823529],
           [0.43137255, 0.2745098 , 0.19215686]],
 
          [[0.37647059, 0.14509804, 0.09411765],
           [0.38431373, 0.15686275, 0.09803922],
           [0.40784314, 0.16470588, 0.09411765],
           ...,
           [0.43529412, 0.28627451, 0.19215686],
           [0.43529412, 0.28627451, 0.19215686],
           [0.43529412, 0.28627451, 0.18823529]],
 
          [[0.34117647, 0.1254902 , 0.09019608],
           [0.34901961, 0.1372549 , 0.09019608],
           [0.38039216, 0.15294118, 0.09019608],
           ...,
           [0.42352941, 0.28235294, 0.17647059],
           [0.42745098, 0.28235294, 0.17647059],
           [0.42745098, 0.28235294, 0.16862745]],
 
          ...,
 
          [[0.24313725, 0.15686275, 0.109803

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [7]:
#write your model here
# create architecture

# define parameters
n_output = 5 # number of classes in case of classification, 1 in case of regression
output_activation = 'softmax' # “softmax” or “sigmoid” in case of classification, “linear” in case of regression

model1 = Sequential()
model1.add(Conv3D(32,(2,2,2),activation='relu',padding='same',input_shape=(15,image_len,image_wid,3)))
model1.add(BatchNormalization())
model1.add(MaxPooling3D(pool_size=(2,2,2)))
model1.add(Dropout(0.25))
model1.add(Conv3D(32,(2,2,2),activation='relu',padding='same'))
model1.add(BatchNormalization())
model1.add(MaxPooling3D(pool_size=(2,2,2)))
model1.add(Dropout(0.25))
model1.add(Conv3D(64,(1,1,1),activation='relu',padding='same'))
model1.add(BatchNormalization())
model1.add(MaxPooling3D(pool_size=(1,2,1)))
model1.add(Dropout(0.25))
model1.add(Conv3D(128,(3,3,3),activation='relu',padding='same'))
model1.add(BatchNormalization())
model1.add(MaxPooling3D(pool_size=(2,2,1)))
model1.add(Dropout(0.25))
model1.add(GlobalAveragePooling3D())
model1.add(Dropout(0.25))
model1.add(Dense(512, activation = 'relu'))
model1.add(BatchNormalization())
model1.add(Dense(256, activation = 'relu'))
model1.add(BatchNormalization())
model1.add(Dense(5,activation=output_activation))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
optimiser = optimizers.Adam(lr=0.01)
model1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 15, 100, 100, 32)  800       
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 100, 100, 32)  128       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 7, 50, 50, 32)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 50, 50, 32)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 7, 50, 50, 32)     8224      
_________________________________________________________________
batch_normalization_2 (Batch (None, 7, 50, 50, 32)     128       
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 3, 25, 25, 32)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name1 = 'model_init_1' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
model_name2 = 'model_init_2' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/' 
model_name3 = 'model_init_3' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'    
if not os.path.exists(model_name1):
    os.mkdir(model_name1)
if not os.path.exists(model_name2):
    os.mkdir(model_name2)
if not os.path.exists(model_name3):
    os.mkdir(model_name3)
        
filepath1 = model_name1 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'
filepath2 = model_name2 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'
filepath3 = model_name3 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint1 = ModelCheckpoint(filepath1, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
checkpoint2 = ModelCheckpoint(filepath2, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
checkpoint3 = ModelCheckpoint(filepath3, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR =  ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)
callbacks_list1 = [checkpoint1, LR]
callbacks_list2 = [checkpoint2, LR]
callbacks_list3 = [checkpoint3, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = int(num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = int(num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [12]:
model1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list1, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size =Source path =  ./Project_data/train ; batch size = 30
Epoch 1/30
 30
23/23 [==============================] - 44s 2s/step - loss: 1.9857 - categorical_accuracy: 0.3652 - val_loss: 6.1084 - val_categorical_accuracy: 0.1333

Epoch 00001: val_loss improved from inf to 6.10845, saving model to model_init_1_2020-09-1416_18_37.126384/model-00001-1.98573-0.36522-6.10845-0.13333.h5
Epoch 2/30
23/23 [==============================] - 37s 2s/step - loss: 1.3308 - categorical_accuracy: 0.3696 - val_loss: 3.1410 - val_categorical_accuracy: 0.2333

Epoch 00002: val_loss improved from 6.10845 to 3.14097, saving model to model_init_1_2020-09-1416_18_37.126384/model-00002-1.33081-0.36957-3.14097-0.23333.h5
Epoch 3/30
23/23 [==============================] - 37s 2s/step - loss: 1.1986 - categorical_accuracy: 0.4319 - val_loss: 2.6078 - val_categorical_accuracy: 0.2500

Epoch 00003: val_loss improved from 3.14097 to 2.60776, saving model to model_init_1_20

MODEL-2 (CNN+RNN)

In [13]:
from keras.applications.vgg16 import VGG16
from tensorflow.keras import Input

n_input=  (image_len,image_wid,3)
n_output = 5 # number of classes in case of classification, 1 in case of regression
output_activation = 'softmax'


cnn = VGG16(include_top=False,input_shape=n_input)

for layer in cnn.layers:
    layer.trainable=False
    
flat1= Flatten()(cnn.layers[-1].output)
cnn = Model(inputs=cnn.inputs, outputs=flat1)
model2= Sequential()
model2.add(TimeDistributed(cnn,input_shape=(15,image_len,image_wid,3)))
model2.add(LSTM(128, dropout=0.25,return_sequences=True))
model2.add(LSTM(128, dropout=0.25))
model2.add(Dense(64, activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dense(5, activation = "softmax"))
optimiser = optimizers.Adam(lr=0.01)
model2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 15, 4608)          14714688  
_________________________________________________________________
lstm_1 (LSTM)                (None, 15, 128)           2425344   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
batch_normalization_7 (Batch (None, 64)                256       
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 325       
Total params: 17,280,453
Trainable params: 2,565,637
Non-trainable params: 14,714,816
________________________________________________________

In [14]:
model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list2, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
23/23 [==============================] - 49s 2s/step - loss: 1.7462 - categorical_accuracy: 0.2101 - val_loss: 2.4471 - val_categorical_accuracy: 0.2000

Epoch 00001: val_loss improved from inf to 2.44708, saving model to model_init_2_2020-09-1416_18_37.126384/model-00001-1.74616-0.21014-2.44708-0.20000.h5
Epoch 2/30
23/23 [==============================] - 37s 2s/step - loss: 1.4591 - categorical_accuracy: 0.2957 - val_loss: 2.0703 - val_categorical_accuracy: 0.2333

Epoch 00002: val_loss improved from 2.44708 to 2.07034, saving model to model_init_2_2020-09-1416_18_37.126384/model-00002-1.45915-0.29565-2.07034-0.23333.h5
Epoch 3/30
23/23 [==============================] - 39s 2s/step - loss: 1.3570 - categorical_accuracy: 0.3609 - val_loss: 2.2794 - val_categorical_accuracy: 0.3083

Epoch 00003: val_loss did not improve from 2.07034
Epoch 4/30
23/23 [==============================] - 39s 2s/step - loss: 1.3060 - categorical_accuracy: 0.3594 - val_loss: 2.6821 - val_categor

### Model -3 (Modified Conv 3d)

In [15]:
#write your model here
# create architecture

# define parameters
n_output = 5 # number of classes in case of classification, 1 in case of regression
output_activation = 'softmax' # “softmax” or “sigmoid” in case of classification, “linear” in case of regression

model3 = Sequential()
model3.add(Conv3D(32,(5,5,5),activation='relu',padding='same',input_shape=(15,image_len,image_wid,3)))
model3.add(BatchNormalization())
model3.add(MaxPooling3D(pool_size=(2,2,2)))
model3.add(Dropout(0.25))

model3.add(Conv3D(32,(3,3,3),activation='relu',padding='same'))
model3.add(BatchNormalization())
model3.add(MaxPooling3D(pool_size=(2,2,2)))
model3.add(Dropout(0.25))

model3.add(Conv3D(64,(2,2,2),activation='relu',padding='same'))

model3.add(Conv3D(128,(1,1,1),activation='relu',padding='same'))
model3.add(BatchNormalization())
model3.add(Dropout(0.25))

model3.add(Conv3D(128,(3,3,3),activation='relu',padding='same'))
model3.add(BatchNormalization())
model3.add(MaxPooling3D(pool_size=(2,2,1)))
model3.add(Dropout(0.25))

model3.add(GlobalAveragePooling3D())
model3.add(Dropout(0.25))
model3.add(Dense(512, activation = 'relu'))
model3.add(BatchNormalization())
model3.add(Dense(256, activation = 'relu'))
model3.add(BatchNormalization())
model3.add(Dense(5,activation=output_activation))
optimiser = optimizers.SGD(lr=0.01)
model3.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model3.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 15, 100, 100, 32)  12032     
_________________________________________________________________
batch_normalization_8 (Batch (None, 15, 100, 100, 32)  128       
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 7, 50, 50, 32)     0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 7, 50, 50, 32)     0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 7, 50, 50, 32)     27680     
_________________________________________________________________
batch_normalization_9 (Batch (None, 7, 50, 50, 32)     128       
_________________________________________________________________
max_pooling3d_6 (MaxPooling3 (None, 3, 25, 25, 32)     0         
__________

In [16]:
model3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list3, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
23/23 [==============================] - 41s 2s/step - loss: 1.7154 - categorical_accuracy: 0.3130 - val_loss: 2.1756 - val_categorical_accuracy: 0.1500

Epoch 00001: val_loss improved from inf to 2.17556, saving model to model_init_3_2020-09-1416_18_37.126384/model-00001-1.71539-0.31304-2.17556-0.15000.h5
Epoch 2/30
23/23 [==============================] - 35s 2s/step - loss: 1.4685 - categorical_accuracy: 0.3725 - val_loss: 4.5877 - val_categorical_accuracy: 0.1167

Epoch 00002: val_loss did not improve from 2.17556
Epoch 3/30
23/23 [==============================] - 34s 1s/step - loss: 1.3881 - categorical_accuracy: 0.4029 - val_loss: 3.8657 - val_categorical_accuracy: 0.1167

Epoch 00003: val_loss did not improve from 2.17556
Epoch 4/30
23/23 [==============================] - 36s 2s/step - loss: 1.2787 - categorical_accuracy: 0.4217 - val_loss: 3.7790 - val_categorical_accuracy: 0.1500

Epoch 00004: val_loss did not improve from 2.17556
Epoch 5/30
23/23 [===============